In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import re
import time
from PIL import Image

import numpy as np

df = pd.DataFrame()
frames = []

In [2]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode


options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2,
)
hands = HandLandmarker.create_from_options(options)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
import os
import copy
path = os.getcwd()

In [4]:
def extract_video(name, target, sequence_id):
    global frames

    # Load an image from file

    image = cv2.imdecode(np.fromfile(name, np.uint8), cv2.IMREAD_UNCHANGED)
    mp_image = mp.Image.create_from_file(name)
    hand_landmarker_result = hands.detect(mp_image)

    # Check if the image was loaded successfully
    if mp_image is not None:
        # Convert the BGR frame to RGB
        rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        if len(hand_landmarker_result.handedness) > 0:
            row_data = {
                "sequence_id": sequence_id,
                "target": target
            }
            
            hand_sides = ["Left", "Right"]
            for idx, landmarks in enumerate(hand_landmarker_result.hand_landmarks):
                detected_pixels = []
                hand_side = hand_sides[idx]
                # Iterate through detected hand landmarks
                for landmark_idx, landmark in enumerate(landmarks):
                    x, y = landmark.x, landmark.y
                    detected_pixels.append([x, y])

                    # Draw circles on the frame
                    cv2.circle(image, (int(x * image.shape[1]), int(y * image.shape[0])), 5, (0, 255, 0), -1)
    

                for i in range(len(detected_pixels)):
                    x, y = detected_pixels[i]
                    row_data[f'x_{hand_side}_hand_{i}'] =  x
                    row_data[f'y_{hand_side}_hand_{i}'] =  y
                

            frames.append(row_data)
        
        # cv2.imshow('Original', image)
    else:
        print("Failed to load image.")

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [5]:
import glob, os
import time
os.chdir("/Users/andreaamaya/Trabajo/megaproyecto_model/data/asl_letters_lensegua_c")

In [6]:
for file in glob.glob("*"):
    if (file.split(".")[-1] != "png"):
        splitted_name = file.split(".")[:-1]
        splitted_name.append(".png")
        renamed_format = "".join(splitted_name)

        im = Image.open(file)
        im.save(renamed_format)

        os.remove(file)


In [7]:
sequence_id = 0
for file in glob.glob("*.png"):
    sequence_id += 1
    name = " ".join(file.split(" ")[:-1])

    if (len(file.split(" ")) == 1):
        name = file.split(".")[0]

    extract_video(file, name, sequence_id)

In [8]:
df = pd.DataFrame(frames)

In [9]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [10]:
len(df)

183

In [11]:
df.head()

,sequence_id,target,x_Left_hand_0,y_Left_hand_0,x_Left_hand_1,y_Left_hand_1,x_Left_hand_2,y_Left_hand_2,x_Left_hand_3,y_Left_hand_3,...,x_Right_hand_16,y_Right_hand_16,x_Right_hand_17,y_Right_hand_17,x_Right_hand_18,y_Right_hand_18,x_Right_hand_19,y_Right_hand_19,x_Right_hand_20,y_Right_hand_20
0,1,o,0.328114,0.495894,0.356607,0.484637,0.376605,0.469754,0.391574,0.459207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,e,0.804299,0.583940,0.750737,0.566927,0.714575,0.514373,0.716878,0.470309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,y,0.277325,0.517538,0.309232,0.492645,0.319768,0.459990,0.314284,0.428498,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,h,0.464587,0.465854,0.447063,0.424897,0.451904,0.387954,0.470445,0.366979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,z,0.201417,0.402654,0.230284,0.383696,0.238570,0.355791,0.222966,0.329582,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
dir_data = "/Users/andreaamaya/Trabajo/megaproyecto_model/data" + "/data_letters.csv"
df.to_csv(dir_data, index=False)

In [13]:
len(df.columns)

86

In [14]:
validation_path = "/Users/andreaamaya/Trabajo/megaproyecto_model/data" + "/validation_letters.csv"

import random
grouped = df.groupby('target')
selected_validation_ids = []
for group_name, group_indices in grouped.groups.items():
    ids = list(set(df.loc[group_indices]['sequence_id'].values))
    if (len(ids) < 0):
        raise ("Error")
    selected_validation_ids.append(random.choice(ids))

validation_df = df[df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
validation_df.to_csv(validation_path, index=False)

In [15]:
train_path = "/Users/andreaamaya/Trabajo/megaproyecto_model/data" + "/train_letters.csv"
train_df = df[~df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
train_df.to_csv(train_path, index=False)